In [27]:
import numpy as np
import pandas as pd
import random
import itertools
from collections import Counter

In [28]:
fh=open('/content/drive/MyDrive/ml/dataset_NB.txt','r')
stop_words = ['the','there','were','may','and','any','are','but','can','did','does','each','else','had','how','has','have','who','yet','you','your','they','their','was','this','for','with','that']
delimiters = ['!','.',',','"']

def clean(fh):
    y=[]
    data=fh.readlines()
    for i in range(len(data)):
        sentence=data[i]
        sentence=sentence.strip()
        data[i]=sentence.lower()
        y.append(int(sentence[-1]))
    clean_data=[]
    words=[]
    
    for sentence in data:
        for i in delimiters:
          sentence=sentence.replace(i,' ')
        x=sentence.split()
        for word in x :
            if(word.isalpha() and word not in stop_words and len(word)>2):
                words.append(word)
        clean_data.append(words)
        words=[]
    
    return clean_data , y

data,y_temp = clean(fh)

In [29]:
x=[i for i in range(1000)]
random.shuffle(x)
newlist, y = [], []
for i in range(0,1000):
  y.append(y_temp[x[i]])
  newlist.append(data[x[i]])

In [30]:
def create_dict(train,y):
  y = np.array(y)
  train = np.array(train)
  spam_t = []
  nspam_t = []
  cnt, ncnt = 0, 0
  for i in range(len(train)):
    if (np.any(y[i] == 0)):
      nspam_t.append(train[i])
      ncnt+=1
    else:
      spam_t.append(train[i])
      cnt+=1
  spam_t1 = tuple(itertools.chain.from_iterable(spam_t))
  nspam_t1 = tuple(itertools.chain.from_iterable(nspam_t))
  spam = Counter(spam_t1)
  nspam = Counter(nspam_t1)
  return spam, nspam, cnt, ncnt



In [35]:
def Naive_Bayes_spam( s,spam,nspam,spam_cnt,nspam_cnt,freq_spam,freq_nspam ):
    p = ((spam[s]/freq_spam)*(spam_cnt/spam_cnt+nspam_cnt)+10)/((spam[s]/freq_spam)*(spam_cnt/spam_cnt+nspam_cnt)+(nspam[s]/freq_nspam)*(nspam_cnt/spam_cnt+nspam_cnt)+20)
    return p

def Naive_Bayes_nspam( s,spam,nspam,spam_cnt,nspam_cnt,freq_spam,freq_nspam ):
    q = ((nspam[s]/ freq_nspam)*(nspam_cnt/spam_cnt+nspam_cnt)+10)/( (spam[s]/ freq_spam)*(spam_cnt/spam_cnt+nspam_cnt)+(nspam[s]/ freq_nspam)*(nspam_cnt/spam_cnt+nspam_cnt)+20)
    return q



In [36]:
lenx = int(len(newlist)/7)
accuracy = []
for i in range(0,7):
  test = []
  y_test = []
  train, y_train = [], []
  if (i!=6):
    test.append(newlist[i*lenx:(i+1)*lenx])
    y_test.append(y[i*lenx:(i+1)*lenx])
  else:
    test.append(newlist[i*lenx:])
    y_test.append(y[i*lenx:])
           
  if (i!=0):
    train.append(newlist[0:i*lenx])
    y_train.append(y[0:i*lenx])
  if (i!=6):
    train.append(newlist[(i+1)*lenx:])
    y_train.append(y[(i+1)*lenx:])
  train = list(itertools.chain.from_iterable(train))
  y_train = list(itertools.chain.from_iterable(y_train))
  spam, nspam, cnt, ncnt = create_dict(train,y_train)
  freq_spam = sum(spam.values())
  freq_nspam = sum(nspam.values())
  
  wrong = 0
  for j in range(len(test[0])):
    nspam_ans = 1
    q=0
    for k in range(0,len(test[0][j])):
      q = Naive_Bayes_spam( test[0][j][k] , spam , nspam, cnt , ncnt , freq_spam ,freq_nspam)
      nspam_ans*= q
    spam_ans = 1
    p=0
    for k in range(0,len(test[0][j])):
      p= Naive_Bayes_nspam( test[0][j][k] , spam , nspam, cnt , ncnt , freq_spam ,freq_nspam)
      spam_ans*= p

    if (spam_ans>nspam_ans and y_test[0][j]==1):
      wrong+=1
    elif (spam_ans<nspam_ans and y_test[0][j]==0):
      wrong+=1
  acc=(1-(wrong/len(y_test[0])))*100
  accuracy.append(acc)

avg_acc = sum(accuracy)/len(accuracy)
print(accuracy)
print(avg_acc)
print(max(accuracy))

[81.69014084507043, 76.05633802816901, 74.64788732394366, 74.64788732394366, 79.5774647887324, 80.28169014084507, 72.97297297297297]
77.12491163195389
81.69014084507043


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  This is separate from the ipykernel package so we can avoid doing imports until
